# **COVID-19 en Perú: Casos Positivos**

![](https://www.cancer.org/es/noticias-recientes/preguntas-comunes-acerca-del-brote-del-nuevo-coronavirus/_jcr_content/par/image.img.jpg/1586550990157.jpg)



### **Objetivo**: *Hacer un análisis exploratorio del dataset, en particular de las Series Temporales*

---

**Casos positivos por COVID-19 - [Ministerio de Salud - MINSA]**, **COVID-19**, **Salud**

Los datos publicados, corresponden al total de casos reportados que dieron positivo al COVID – 19, por departamento, provincia y distrito.  Adicionalmente encontraran datos que son de interés nacional, los mismos que se ponen a disposición de la ciudadanía. **Nota:** Desde el **02/06** lo casos por PR no incluye las IPRESS privadas que realizan tamizaje a trabajadores de empresas en el marco de la reactivación económica, debido a que el objetivo de este tamizaje NO permite identificar casos nuevos en personas sospechosas

**Fuente:** Instituto Nacional de Salud y Centro Nacional de Epidemiologia, prevención y Control de Enfermedades – MINSA.

**Frecuencia de actualización:** La información se actualiza diariamente





In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### **Obtener Datos**

La iniciativa del Gobierno al respecto de los datos abiertos de COVID-19 se encuentra en https://www.datosabiertos.gob.pe/dataset/casos-positivos-por-covid-19-ministerio-de-salud-minsa

In [ ]:
!wget -O positivos_covid.csv https://cloud.minsa.gob.pe/s/Y8w3wHsEdYQSZRp/download

In [ ]:
ls

### **Leer Datos**

In [ ]:
df = pd.read_csv('positivos_covid.csv', sep=";")

df.head()

In [ ]:
df.head()

In [ ]:
df = pd.read_csv('positivos_covid.csv', sep=";", parse_dates=['FECHA_CORTE', 'FECHA_RESULTADO'])
#df = pd.read_csv('positivos_covid.csv', sep = ";")
df.head()

In [ ]:
df.info()

In [ ]:
df.groupby(by='SEXO').size()

In [ ]:
#df.groupby(by='DISTRITO').count()

In [ ]:
df.head()

In [ ]:
sns.boxplot(data = df,x='EDAD')

In [ ]:
# Boxplot de la edad antes del cleaning
sns.boxplot(data = df[df['EDAD'] > 10], x = 'EDAD')

In [ ]:
# Data Cleaning
df.loc[df['EDAD'] < 0, 'EDAD'] = df[df['EDAD'] < 0]['EDAD'].map(lambda x: x * -1)

In [ ]:
sns.boxplot(data = df,x='EDAD')

In [ ]:
df[['FECHA_RESULTADO', 'EDAD']].groupby('FECHA_RESULTADO').count()

In [ ]:
# Graficos
df[['FECHA_RESULTADO', 'EDAD']].groupby('FECHA_RESULTADO').median().plot(figsize=(30, 10))
plt.show()

In [ ]:
# Graficos
df[['FECHA_RESULTADO', 'EDAD']].groupby('FECHA_RESULTADO').count().plot(figsize=(30, 10))
plt.show()

In [ ]:
# Boxplot de la edad despues del cleaning
sns.boxplot(data = df, x = 'EDAD')

In [ ]:
sns.countplot(data = df, x = "METODODX", hue = "SEXO")

In [ ]:
# Hacer un Group by y un Pivot
df_state = df[['METODODX', 'FECHA_RESULTADO', 'EDAD']].groupby(['FECHA_RESULTADO', 'METODODX']).median()

In [ ]:
df_state.pivot_table(index = ["METODODX"], columns = ['FECHA_RESULTADO'], values = "EDAD")

# **Ejercicios**

### Pregunta 1

¿Cuántas pruebas fueron ejecutadas?

In [ ]:
# CODIGO GOES HERE
df.shape[0]

### Pregunta 2

Verifique si existen duplicados ¿qué porcentaje son registros únicos? Elimine los datos duplicados (nota: es necesario definir el tipo de criterio). Identifique que registros tienen más duplicados ¿cuántos duplicados tiene el registro que más se repite?

In [ ]:
sns.heatmap(df.isna())

In [ ]:
df.isnull().sum()

In [ ]:
df.FECHA_RESULTADO = df.FECHA_RESULTADO.fillna('2021-02-10')
df.EDAD  = df.EDAD.fillna(df.EDAD.median())


In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.pivot_table(index=list(df.columns), aggfunc='size')

In [ ]:
# CODE GOES HERE
dups_values = df.pivot_table(index=list(df.columns), aggfunc='size')
print(df.shape[0] - dups_values.shape[0] )

In [ ]:
len(df.drop_duplicates())/len(df)

### Pregunta 3

Contar la cantidad de nulos por columna

In [ ]:
# CODE GOES HERE
df.isnull().sum()

¿Qué proporción de nulos tiene cada columna?

In [ ]:
# CODE GOES HERE
(df.isnull().sum())/len(df.index)*100


### Pregunta 4

¿Cuál es la cantidad de casos positivos por cada **tipo de prueba**?

In [ ]:
# CODE GOES HERE
df.groupby('METODODX').UUID.count()


### Pregunta 5

¿Cuál es la cantidad de casos positivos por **departamento** y **tipo de prueba**?

In [ ]:
df[['DEPARTAMENTO','METODODX', 'UUID']].groupby(['DEPARTAMENTO','METODODX']).count()

### Pregunta 6

¿Cuál fue la persona de mayor edad que dió positivo a **COVID-19**?

In [ ]:
# CODE GOES HERE
df.EDAD.max()

In [ ]:
df.query("EDAD == @df.EDAD.max()")


In [ ]:
df.loc[df.EDAD == df.EDAD.max()][:]


### Pregunta 7

Ordene los distritos de Lima por cantidad de casos

In [ ]:
# CODE GOES HERE
df[['DISTRITO', 'FECHA_CORTE']].groupby('DISTRITO').count().rename(columns = {'FECHA_CORTE': 'COUNT'})['COUNT'].sort_values()

In [ ]:
pd.DataFrame(df[df['DEPARTAMENTO'] == "LIMA"].value_counts('DISTRITO')).rename(columns = {0: 'COUNT'})

In [ ]:
df[df.DEPARTAMENTO == 'LIMA'][['DISTRITO','UUID']].groupby(['DISTRITO']).count().reset_index().sort_values('UUID',ascending=False)


### Pregunta 8

¿Cuál es el caso más antiguo?

In [ ]:
# CODE GOES HERE
df.min()

### Pregunta 9

¿Cuál es el caso más reciente?

In [ ]:
df.info()

In [ ]:
# CODE GOES HERE
df.max()


### Pregunta 10

Muestre un histograma de casos por tiempo ¿qué concluye sobre los picos y depresiones en la serie temporal? Muestre también el gráfico de casos acumulados.

In [ ]:
# CODE GOES HERE
df.groupby('FECHA_RESULTADO').size().plot()

In [ ]:
# CODE GOES HERE
df.groupby('FECHA_RESULTADO').size().cumsum().plot()


### Pregunta 11

Muestre la serie temporal procesada realizando un moving average

In [ ]:
# CODE GOES HERE
df.groupby('FECHA_RESULTADO').size().rolling(7, center=False).mean().plot()

### Pregunta 12

Realice esto mismo y muestre 3 distritos en una Departamento/Provincia, por ejemplo en Lima podría ser `['SAN MIGUEL', 'MAGDALENA DEL MAR', 'PUENTE PIEDRA']`

In [ ]:
# CODE GOES HERE
df[df['DISTRITO'] == "SAN MIGUEL"].groupby('FECHA_RESULTADO').size().rolling(7, center=False).mean().plot()
df[df['DISTRITO'] == "PUENTE PIEDRA"].groupby('FECHA_RESULTADO').size().rolling(7, center=False).mean().plot()
df[df['DISTRITO'] == "MAGDALENA DEL MAR"].groupby('FECHA_RESULTADO').size().rolling(7, center=False).mean().plot()
plt.legend(['SAN MIGUEL', 'PUENTE PIEDRA', 'MAGDALENA DEL MAR'])
plt.title("CASOS POR DISTRITO SM PT MDM")

In [ ]:
# CODE GOES HERE
df[df['DISTRITO'] == "EL TAMBO"].groupby('FECHA_RESULTADO').size().rolling(7, center=False).mean().plot()
df[df['DISTRITO'] == "HUANCAYO"].groupby('FECHA_RESULTADO').size().rolling(7, center=False).mean().plot()
df[df['DISTRITO'] == "CHILCA"].groupby('FECHA_RESULTADO').size().rolling(7, center=False).mean().plot()
plt.legend(['EL TAMBO', 'HUANCAYO', 'CHILCA'])
plt.title("CASOS POR DISTRITO EL TAMBO, HUANCAYO, CHILCA")